<a href="https://colab.research.google.com/github/junkyuhufs/Practice/blob/main/prepro_wos_bib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##wos의 bib file중 article만 선택 (12-11-2024)

In [ ]:
# 코랩에서 파일 업로드
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 가져오기
input_file_name = list(uploaded.keys())[0]
output_file_name = 'filtered_articles.bib'

# `@article` 항목 필터링 함수
def filter_bib_entries_by_type(input_file, output_file, entry_type="article"):
    with open(input_file, 'r', encoding='utf-8') as file:
        entries = []
        current_entry = []
        include_entry = False
        for line in file:
            if line.strip() == "}":  # End of a BibTeX entry
                current_entry.append(line)
                if include_entry:
                    entries.append("".join(current_entry))
                current_entry = []
                include_entry = False
            else:
                current_entry.append(line)
                # Check if the entry type matches (e.g., "@article{")
                if line.strip().lower().startswith(f"@{entry_type.lower()}"):
                    include_entry = True

    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(entries)

# 필터링 실행
filter_bib_entries_by_type(input_file_name, output_file_name)

# 결과 파일 다운로드
files.download(output_file_name)


##wos의 bib file중 참고문헌 중 무명저자 삭제

In [ ]:
# 코랩에서 파일 업로드
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 가져오기
input_file_name = list(uploaded.keys())[0]
output_file_name = 'cleaned_cr_entries.bib'

# CR 섹션을 청소하는 함수 정의
def clean_cr_references(input_file, output_file, key="CR", exclude_value="Anonymous"):
    cleaned_entries = []
    with open(input_file, 'r', encoding='utf-8') as file:
        current_entry = []
        inside_cr = False
        cr_lines = []

        for line in file:
            if line.strip() == "}":  # End of a BibTeX entry
                if inside_cr:
                    # Filter and reconstruct the CR section
                    filtered_cr_lines = [
                        cr_line for cr_line in cr_lines if exclude_value not in cr_line
                    ]
                    if filtered_cr_lines:
                        cleaned_entries.append(f"{key} = {{{'; '.join(filtered_cr_lines)}}},\n")
                cleaned_entries.append("".join(current_entry))
                current_entry = []
                inside_cr = False
                cr_lines = []
            else:
                if f"{key} =" in line:  # Detect CR section
                    inside_cr = True
                    cr_lines = [line.split("=", 1)[1].strip().strip("{}").split("; ")]
                elif inside_cr and line.strip().endswith(","):  # Continuation of CR section
                    cr_lines.append(line.strip().strip(",").strip())
                else:
                    if inside_cr:
                        # End CR section if a non-continuation line is encountered
                        filtered_cr_lines = [
                            cr_line for cr_line in cr_lines if exclude_value not in cr_line
                        ]
                        if filtered_cr_lines:
                            cleaned_entries.append(f"{key} = {{{'; '.join(filtered_cr_lines)}}},\n")
                        inside_cr = False
                        cr_lines = []
                    cleaned_entries.append(line)

    # Write cleaned entries to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(cleaned_entries)

# 함수 실행하여 파일 정리
clean_cr_references(input_file_name, output_file_name)

# 결과 파일 다운로드
files.download(output_file_name)
